In [1]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
import os
import re
import json

#cwd = dir_path = os.path.dirname(os.path.realpath(__file__)) 
# bidsdir = os.path.join('%s/../bids'%(cwd))
cwd='/ZPOOL/data/projects/rf1-mbme-pilot/code'
bidsdir='/ZPOOL/data/projects/rf1-mbme-pilot/bids/'

print(bidsdir)

/ZPOOL/data/projects/rf1-mbme-pilot/bids/


In [2]:
# Read events files, find unique sub IDs

event_files=[os.path.join(root,f) for root,dirs,files in os.walk(bidsdir) for f in files if f.endswith('events.tsv')]
#print(event_files)

data=[]
for file in event_files:
	sub='sub-'+re.search('func/sub-(.*)_task',file).group(1)
	acq=re.search('_acq-(.*)_events',file).group(1)
	tmp_df=pd.read_csv(file,sep='\t')
#	print(tmp_df)
	if tmp_df.shape[0]>0:
		#print(sub,acq)
		tmp_df['sub']=sub
		tmp_df['acq']=acq
		data.append(tmp_df)
events_df=pd.concat(data)

#print(events_df)

print("Unique subs:")
print(events_df['sub'].unique())


Unique subs:
['sub-10422' 'sub-10186' 'sub-10382' 'sub-10150' 'sub-10391' 'sub-10296'
 'sub-10590sp' 'sub-10741sp' 'sub-10043' 'sub-10723sp' 'sub-10198'
 'sub-10223' 'sub-10659sp' 'sub-10015' 'sub-10318' 'sub-10185'
 'sub-10803sp' 'sub-10603sp' 'sub-10320' 'sub-10054' 'sub-10008'
 'sub-10589sp' 'sub-10438' 'sub-10234' 'sub-10137' 'sub-10030'
 'sub-10608sp' 'sub-10130' 'sub-10078' 'sub-10108' 'sub-10024' 'sub-10059'
 'sub-10154' 'sub-10188' 'sub-10166' 'sub-10085' 'sub-10640sp'
 'sub-10644sp' 'sub-10416' 'sub-10026' 'sub-10606sp' 'sub-10014'
 'sub-10069' 'sub-10035' 'sub-10203' 'sub-10094' 'sub-10007' 'sub-10303'
 'sub-10142' 'sub-12042' 'sub-10080' 'sub-10777sp' 'sub-10221'
 'sub-10691sp' 'sub-10017' 'sub-10363' 'sub-10041' 'sub-10003' 'sub-10074'
 'sub-10136' 'sub-10319' 'sub-10010' 'sub-10321' 'sub-10690sp' 'sub-10125'
 'sub-10028']


In [19]:
# Find unique acquisitions for each sub & append relevant info to exclusions dataframe

data=[]
for sub in events_df['sub'].unique():
    #print(sub)
    for acq in events_df['acq'].unique():
        
        # Check for abs motion file (generated by run_motioneval.sh)
        fname='%s/../derivatives/fsl/mcflirt/%s/%s/_abs.rms'%(cwd,sub,acq)
        print(fname)
        if os.path.exists(fname):
            absolute=np.loadtxt(fname)
            
            file="/ZPOOL/data/projects/rf1-mbme-pilot/derivatives/mriqc/%s/func/%s_task-sharedreward_acq-%s_echo-2_bold.json"%(sub,sub,acq)
            if os.path.exists(file):
                f = open(file)
            else:
                file="/ZPOOL/data/projects/rf1-mbme-pilot/derivatives/mriqc/%s/func/%s_task-sharedreward_acq-%s_bold.json"%(sub,sub,acq)
                if os.path.exists(file):
                    f = open(file)
                else:
                    print(file+" does not exist")

            #print(f)
            
            # Returns JSON object as a dictionary
            FD = json.load(f)
            f.close
            FD=FD['fd_mean']
            print(FD)
            FD=np.loadtxt('%s/../derivatives/fsl/mcflirt/%s/%s/_rel.rms'%(cwd,sub,acq))

            row=[sub,acq,events_df[(events_df['sub']==sub)&(events_df['acq']==acq)]['trial_type'].str.count('miss_decision').sum(),absolute.max(),FD]
            #row=[sub,acq,
            #    events_df[(events_df['sub']==sub)&(events_df['acq']==acq)]['trial_type'].str.count('miss_decision').sum(),
            #    FD]
            data.append(row)

# Generate exclusions df
exclusions_df=pd.DataFrame(data=data,columns=['sub','acq','TrialCount_misses','Max_Abs_motion','FD_mean'])
#exclusions_df['FD_exclusion']=exclusions_df['FD_mean']>0.5
#exclusions_df['ABS_exclusion']=exclusions_df['Max_Abs_motion']>1.35
#exclusions_df['Beh_TrialExclusion']=exclusions_df['TrialCount_misses']>27
#fd_df=exclusions_df[exclusions_df['FD_mean'] > 0.5]
print(exclusions_df)

/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10422/mb6me4/_abs.rms
0.11748862966049424
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10422/mb1me1/_abs.rms
0.11790981915470071
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10422/mb3me4/_abs.rms
0.17063210517665084
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10422/mb3me1/_abs.rms
0.18212286985468443
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10422/mb1me4/_abs.rms
0.19790307797817064
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10422/mb6me1/_abs.rms
0.14001193810836773
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10422/mb3me3ip0/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10422/mb3me3/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10422/mb3me4fa50/_abs.rms
/ZPOOL/data/projects/rf1-mb

/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10043/mb3me4/_abs.rms
0.11443271211811677
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10043/mb3me1/_abs.rms
0.1691472154953652
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10043/mb1me4/_abs.rms
0.0978879778895726
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10043/mb6me1/_abs.rms
0.1065505793551644
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10043/mb3me3ip0/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10043/mb3me3/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10043/mb3me4fa50/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10043/mb2me4/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10043/mb3me1fa50/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflir

/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10320/mb1me1/_abs.rms
0.06958544284076845
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10320/mb3me4/_abs.rms
0.09593662055436229
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10320/mb3me1/_abs.rms
0.12123918903834142
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10320/mb1me4/_abs.rms
0.08166621133643363
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10320/mb6me1/_abs.rms
0.09130242809014931
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10320/mb3me3ip0/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10320/mb3me3/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10320/mb3me4fa50/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10320/mb2me4/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../der

0.10018748312852832
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10030/mb3me1/_abs.rms
0.26313585388118793
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10030/mb1me4/_abs.rms
0.0807499566795942
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10030/mb6me1/_abs.rms
0.11849397168415993
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10030/mb3me3ip0/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10030/mb3me3/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10030/mb3me4fa50/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10030/mb2me4/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10030/mb3me1fa50/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10608sp/mb6me4/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mc

/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10154/mb1me4/_abs.rms
0.14873597974796313
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10154/mb6me1/_abs.rms
0.10106914898464009
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10154/mb3me3ip0/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10154/mb3me3/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10154/mb3me4fa50/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10154/mb2me4/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10154/mb3me1fa50/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10188/mb6me4/_abs.rms
0.0984974768605606
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10188/mb1me1/_abs.rms
0.1130674075603807
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcfli

/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10026/mb6me1/_abs.rms
0.12635400021633814
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10026/mb3me3ip0/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10026/mb3me3/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10026/mb3me4fa50/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10026/mb2me4/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10026/mb3me1fa50/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10606sp/mb6me4/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10606sp/mb1me1/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10606sp/mb3me4/_abs.rms
0.1892106102915377
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10606sp/mb3me1/_abs.rms
/Z

0.34963441758775976
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10007/mb3me3ip0/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10007/mb3me3/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10007/mb3me4fa50/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10007/mb2me4/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10007/mb3me1fa50/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10303/mb6me4/_abs.rms
0.09328216093271541
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10303/mb1me1/_abs.rms
0.0998475903105378
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10303/mb3me4/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10303/mb3me1/_abs.rms
0.1452715351542941
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcfli

/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10017/mb3me4/_abs.rms
0.09486105558963757
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10017/mb3me1/_abs.rms
0.11951228259124183
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10017/mb1me4/_abs.rms
0.07354440206775066
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10017/mb6me1/_abs.rms
0.1135909527310409
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10017/mb3me3ip0/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10017/mb3me3/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10017/mb3me4fa50/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10017/mb2me4/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10017/mb3me1fa50/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcfl

/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10010/mb6me1/_abs.rms
0.16642918840247878
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10010/mb3me3ip0/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10010/mb3me3/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10010/mb3me4fa50/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10010/mb2me4/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10010/mb3me1fa50/_abs.rms
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10321/mb6me4/_abs.rms
0.09041744591181872
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10321/mb1me1/_abs.rms
0.1121153972780254
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcflirt/sub-10321/mb3me4/_abs.rms
0.08045414820905325
/ZPOOL/data/projects/rf1-mbme-pilot/code/../derivatives/fsl/mcfl

In [ ]:
results=exclusions_df.groupby(by='sub').sum().reset_index().rename(columns={"TrialCount_misses": "TotalCount_misses"})
print(results)
results['Beh_TotalExclusion']=results['TotalCount_misses']>81
results=results[['sub','TotalCount_misses','Beh_TotalExclusion']]

In [ ]:
exclusions_df.merge(results,on='sub')
exclusions_df.to_csv('../derivatives/exclusions.csv', index=False)